In [1]:
import requests
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
url = 'https://timeseries.geomar.de/soop/FROST-Server/v1.1/'
t_box = 'Things(3)'

def get_content(url):
    response = requests.get(url)
    content = response.text
    return json.loads(content)

content = get_content(url + t_box)
print(json.dumps(content, indent=4))


{
    "@iot.selfLink": "https://timeseries.geomar.de/soop/FROST-Server/v1.1/Things(8)",
    "@iot.id": 8,
    "name": "SOOP_Fridtjof_Nansen",
    "description": "Hurtigruten Cruise Vessel MS Fridtjof Nansen (https://www.hurtigruten.com/de-de/expeditions/schiffe/fridtjof-nansen/)",
    "properties": {
        "@id": "https://registry.o2a-data.de/rest/v2/items/41707ad6-71ba-4c89-953d-4946b91e02fc",
        "metadata": "https://registry.o2a-data.de/rest/v2/items/%7b11658%7d?with=all",
        "identifier": "41707ad6-71ba-4c89-953d-4946b91e02fc",
        "platform_id": "41707ad6-71ba-4c89-953d-4946b91e02fc"
    },
    "Locations@iot.navigationLink": "https://timeseries.geomar.de/soop/FROST-Server/v1.1/Things(8)/Locations",
    "HistoricalLocations@iot.navigationLink": "https://timeseries.geomar.de/soop/FROST-Server/v1.1/Things(8)/HistoricalLocations",
    "Datastreams@iot.navigationLink": "https://timeseries.geomar.de/soop/FROST-Server/v1.1/Things(8)/Datastreams",
    "MultiDatastreams@iot

In [3]:
datastream_url = content['Datastreams@iot.navigationLink']
content_datastream = get_content(datastream_url)
#print(json.dumps(content_datastream, indent=4, ensure_ascii=False))
observation_url = content_datastream['value'][0]["Observations@iot.navigationLink"]

In [ ]:
# Base URL for observations of Datastream(3), sorted by phenomenonTime
params = {
    "$top": 1,  # Limit to 1000 observations per page
    "$orderby": "phenomenonTime asc"  # Sort by phenomenonTime in ascending order
    
}

# List to store all observations
all_observations = []
next_link = observation_url

# Loop through all pages until there's no @iot.nextLink
while next_link:
    response = requests.get(next_link, params=params if next_link == observation_url else None)
    
    if response.status_code == 200:
        data = response.json()
        all_observations.extend(data["value"])  # Add observations to the list
        
        # Check for pagination link
        next_link = data.get("@iot.nextLink")  # Automatically handles pagination
    else:
        print(f"Error: {response.status_code}")
        break

# Display the first 5 observations
for obs in all_observations[:5]:
    print(f"Time: {obs['phenomenonTime']}, Result: {obs['result']}")

print(f"Total Observations Retrieved: {len(all_observations)}")

In [37]:
df_obs = pd.DataFrame(all_observations)
display(df_obs.head(3))
display(df_obs.info())

""


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


None

In [38]:
# convert phenomenonTime and resultTime to datetime
df_obs["phenomenonTime"] = pd.to_datetime(df_obs["phenomenonTime"])
df_obs["resultTime"] = pd.to_datetime(df_obs["resultTime"])

# convert result to float
df_obs["result"] = df_obs["result"].astype(float)



KeyError: 'phenomenonTime'

In [17]:
# Plot using Plotly
fig = px.line(
    df_obs, 
    x='phenomenonTime', 
    y='result', 
    title='Observations Over Time',
    labels={'phenomenonTime': 'Time', 'result': 'Measurement Value'},
    line_shape='spline'  # Smooth line
)

fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Result',
    template='plotly_white',  # Optional: dark theme
    hovermode='x unified'    # Unified hover for better interactivity
)

# Show the plot
fig.show()

In [18]:
import plotly.express as px

# Plot with Plotly
fig = px.line(
    df_obs, 
    x='phenomenonTime', 
    y='result', 
    title='Observations Over Time',
    labels={'phenomenonTime': 'Time', 'result': 'Measurement Value'},
    line_shape='spline'
)

# Add interactive date range slider and buttons
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Result',
    template='plotly_white',
    hovermode='x unified',
    
    # Add range slider
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="Last 7 Days", step="day", stepmode="backward"),
                dict(count=1, label="Last Month", step="month", stepmode="backward"),
                dict(count=6, label="6M", step="month", stepmode="backward"),
                dict(step="all")  # Show all data
            ])
        ),
        rangeslider=dict(visible=True),  # Enable the range slider
        type="date"
    )
)

# Show the plot
fig.show()
